<h1 style="text-align: center; font-size: 3em;">
  Introducción a la biblioteca <code>macti_lib--eval</code>
</h1>

<code>macti_lib</code> es una biblioteca diseñada para facilitar la **evaluación automática**, en particular **eval**, de ejercicios dentro de entornos como Google Colab o Jupyter Notebook. La biblioteca está compuesta principalmente por dos elementos clave —<strong>FileAnswer</strong> y <strong>Quiz</strong>— con los cuales podemos capturar las respuestas de los estudiantes, almacenarlas en archivos Parquet y dar retroalimentación inmediata, detallada y configurable, todo ello sin salir del entorno.

## ¿Para qué sirve?

- Automatizar la evaluación de tareas prácticas  
- Dar retroalimentación personalizada  
- Aumentar la objetividad y eficiencia en la revisión  
- Permitir múltiples tipos de respuestas (desde texto hasta estructuras complejas)

## ¿Qué tipos de evaluación soporta?

| Tipo                    | Método              | Ejemplo                             |
|-------------------------|---------------------|--------------------------------------|
| Opción múltiple         | `eval_option()`     | `'a'`, `'b'`, `'c'`, ...             |
| Expresiones simbólicas  | `eval_expression()` | `x**2`, `sin(x)`                     |
| Valores numéricos       | `eval_numeric()`    | `3.14`, `[1, 2, 3]`, `2 + 3j`        |
| Estructuras de datos    | `eval_datastruct()` | `['a', 'b']`, `('x', 'y')`, `{1,2}`  |
| Diccionarios            | `eval_dict()`       | `{1: 2.5}`, `{'k1': 'rojo'}`         |

Durante este notebook conocerás cómo funciona internamente la biblioteca, con breves ejemplos de su uso.


<p>
La biblioteca <code>macti_lib</code> está compuesta por dos clases principales: <code>FileAnswer</code> y <code>Quiz</code>, cada una con sus características particulares que se describen a continuación:
</p>

<ul>
  <li>
    La clase <strong>FileAnswer</strong> se encarga de <em>escribir</em> respuestas y comentarios en memoria, para luego almacenarlas en archivos de texto.
    <ul style="font-size: 1em; line-height: 1.5;">
      <li>Almacena respuestas numéricas, simbólicas, listas, diccionarios y más.</li>
      <li>Convierte arreglos <code>numpy</code> a vectores 1D y descompone diccionarios en longitudes, claves y valores.</li>
      <li>Genera dos archivos por “quiz”: uno con respuestas (<code>.__ans_<em>qnum</em></code>) y otro con retroalimentación (<code>.__fee_<em>qnum</em></code>).</li>
    </ul>
  </li>

  <li>
    La clase <strong>Quiz</strong> es responsable de <em>leer</em> las respuestas almacenadas y de <em>evaluar</em> las respuestas del alumno.
    Ofrece métodos para distintos tipos de preguntas:
    <ul style="font-size: 1em; line-height: 1.5;">
      <li><code>eval_option</code>: opción múltiple, compara cadenas.</li>
      <li><code>eval_expression</code>: expresiones simbólicas con <code>sympy</code>.</li>
      <li><code>eval_numeric</code>: números y arreglos numéricos con tolerancia.</li>
      <li><code>eval_datastruct</code>: listas y tuplas (con o sin orden).</li>
      <li><code>eval_dict</code>: validación de diccionarios (tamaños, claves y valores).</li>
    </ul>
    Además, utiliza <code>colorama</code> para colorear la salida (verde para correcto, rojo para errores) y ofrece distintos niveles de verbosidad.
  </li>
</ul>


<p>
  <strong>¿Cómo empezar?</strong>
</p>

<ol style="font-size: 1em; line-height: 1.5;">
  <li>
    Importa <code>evaluation</code> desde el repositorio <code>macti_lib</code>.
  </li>
  <li>
    En nuestro notebook, creamos un objeto <code>FileAnswer()</code>.  
    Utiliza el método <code>write(enum, respuesta, feedback)</code> para almacenar cada respuesta junto con su retroalimentación.  
    Al finalizar, se guarda todo en archivos parquet usando el método <code>to_file(&lt;número_del_quiz&gt;)</code>.
  </li>
  <li>
    Posteriormente, creamos un objeto <code>Quiz(&lt;número_del_quiz&gt;)</code> para cargar las respuestas almacenadas.  
    Utilizamos los métodos <code>eval_*</code> (como <code>eval_option</code>, <code>eval_expression</code>, entre otros) para evaluar las respuestas de los estudiantes.
  </li>
  <li>
    Ajustamos la propiedad <code>verb</code> (niveles 0, 1 o 2) para definir el nivel de detalle en la retroalimentación proporcionada a los alumnos:
    <ul style="font-size: 1em; line-height: 1.5;">
      <li><strong>0:</strong> Solo indica si la respuesta es correcta o incorrecta.</li>
      <li><strong>1:</strong> Muestra comentarios adicionales para ayudar al alumno.</li>
      <li><strong>2:</strong> Proporciona información detallada sobre el error o la respuesta correcta.</li>
    </ul>
  </li>
</ol>

<p style="font-size: 0.9em; margin-top: 2em;">
  Con <code>macti_lib</code>, la evaluación en notebooks se vuelve más sencilla y práctica para poder realizar evaluaciones y ejercicios de una manera más eficiente.
</p>

---


## Flujo de trabajo general para hacer evaluaciones

Para aprovechar las funciones de evaluación automática, es importante conocer el flujo de trabajo y los pasos necesarios para llevar a cabo la evaluación de manera exitosa.

A continuación se muestran los dos pasos básicos para usar **macti_lib** en un notebook:

---  


1. **Registrar las respuestas correctas**  
2. **Evaluar la respuesta del estudiante**

### 1. Registrar respuestas correctas

```python
from macti.eval.file_answer import FileAnswer

# 1.1 Inicializamos FileAnswer para almacenar:
#    - enum: identificador del ejercicio
#    - ans : respuesta correcta
#    - feed: retroalimentación si el estudiante falla
file_answer = FileAnswer()

# 1.2 Almacenamos el ejercicio y la respuesta correcta
#    file_answer.write(enum: str, ans: Any, feed: str)
#      • enum (str): identificador del ejercicio (p. ej. '1', '2', …)
#      • ans  (Any): respuesta correcta (p. ej. 'b', número, lista, etc.)
#      • feed (str): mensaje de ayuda en caso de respuesta incorrecta
file_answer.write(
    '1',                              # enum: identificador del ejercicio
    'b',                              # ans : respuesta correcta
    'Recuerda revisar las opciones...'# feed: sugerencia si el alumno falla
)

# 1.3 – Exportar todo a disco en formato Parquet
#    • .ans/<topic>/.__ans_quiz01   (archivo de respuestas)
#    • .ans/<topic>/.__fee_quiz01   (archivo de feedback)
file_answer.to_file('quiz01')

### 2. Evaluar la respuesta del estudiante

Para comprobar las respuestas con **macti_lib**, instanciamos `Quiz` y usamos el método `eval_option`:

```python
from macti.eval.quiz import Quiz

# 2.1 Inicializar Quiz:
#    - qnum   (str): debe coincidir con el valor de file_answer.to_file('quiz01')
#    - server (str): ubicación de los archivos Parquet ('local' o 'hub')
quiz = Quiz(qnum='quiz01', server='local')

# 2.2 Evaluar la respuesta del alumno:
#    - enum (str): identificador del ejercicio ('1')
#    - ans  (str): respuesta proporcionada por el estudiante ('b')
quiz.eval_option('1', 'b')


---
## Ejemplo práctico de uso de **macti_lib**

A continuación, se explica paso a paso en qué consiste este fragmento de código y cómo aprovecha la biblioteca:



In [14]:
# Importamos los módulos necesarios para poder hacer uso de la biblioteca de MACTI
from macti.eval import FileAnswer, Quiz 
import sympy as sy
import numpy as np
import pandas as pd

In [15]:
# 1) Inicializar el FileAnswer (recolector de respuestas) 
file_answer = FileAnswer()

# 2) Registrar una pregunta de opción múltiple:
#    • '1' → identificador del ejercicio.
#    • 'c' → respuesta correcta.
#    • 'Revisa tus opciones.' → retroalimentación en caso de error.
file_answer.write('1', 'c', 'Revisa tus opciones.')

# 3) Registrar una pregunta de expresión simbólica:
#    • Definimos el símbolo x con SymPy.
#    • Guardamos "x**2" como cadena para evaluación simbólica.
x = sy.Symbol('x')
file_answer.write('2', str(x**2), 'Deriva correctamente.')

# 4) Registrar una pregunta de valor numérico:
#    • '3' → identificador del ejercicio.
#    • 3.1416 → valor correcto.
#    • 'Calcula con mayor precisión.' → retroalimentación si la aproximación es imprecisa.
file_answer.write('3', 3.1416, 'Calcula con mayor precisión.')

# 5) Registrar una pregunta de estructura de datos (lista):
#    • '4' → identificador del ejercicio.
#    • ['a', 'b', 'c'] → lista esperada (el orden importa).
#    • 'Revisa el orden.' → retroalimentación si el orden difiere.
file_answer.write('4', ['a', 'b', 'c'], 'Revisa el orden.')

# 6) Registrar una pregunta de diccionario:
#    • '5' → identificador del ejercicio.
#    • {1: 2.0, 2: 4.0} → diccionario con claves y valores correctos.
#    • 'Revisa claves y valores.' → retroalimentación si falta o está mal algún par.
file_answer.write('5', {1: 2.0, 2: 4.0}, 'Revisa claves y valores.')

# 7) Exportar todas las respuestas y retroalimentación a un archivo de texto:
#    • crea ".ans/<topic>/.__ans_intro_demo" y ".ans/<topic>/.__fee_intro_demo".
file_answer.to_file('intro_demo')


El directorio :/home/jovyan/MACTI_LIB/.ans/Eval/ ya existe
Respuestas y retroalimentación almacenadas.


In [16]:
# 2) Evaluar las respuestas con Quiz (suponiendo que Quiz y sympy ya estén importados)

# 2.1 Inicializar Quiz con los archivos generados por FileAnswer
quiz = Quiz(qnum='intro_demo', server='local')  # Carga respuestas y retroalimentación desde Parquet

# 2.2 Opción múltiple (ejercicio '1')
quiz.eval_option('1', 'c')
#    • Compara la respuesta del alumno ('c') con la correcta registrada.

# 2.3 Expresión simbólica (ejercicio '2')
quiz.eval_expression('2', sy.sympify('x**2'))
#    • Convierte 'x**2' a expresión de SymPy y la compara simbólicamente.

# 2.4 Valor numérico (ejercicio '3')
quiz.eval_numeric('3', 3.1416)
#    • Verifica que el valor entregado esté dentro de la tolerancia establecida.

# 2.5 Estructura de datos: lista ordenada (ejercicio '4')
quiz.eval_datastruct('4', ['a', 'b', 'c'])
#    • Comprueba que la lista coincida exactamente y en el orden correcto.

# 2.6 Diccionario numérico (ejercicio '5')
quiz.eval_dict('5', {1: 2.0, 2: 4.0}, numeric=True)
#    • Valida que las claves y valores coincidan con tolerancia numérica.


――――――――――――――――――――――――――――――――――――――――
1 | Tu respuesta: c, es correcta.
――――――――――――――――――――――――――――――――――――――――
――――――――――――――――――――――――――――――――――――――――
2 | Tu respuesta:


x**2

es correcta.
――――――――――――――――――――――――――――――――――――――――
――――――――――――――――――――――――――――――――――――――――
3 | Tu resultado es correcto.
――――――――――――――――――――――――――――――――――――――――
――――――――――――――――――――――――――――――――――――――――
4 | Tu resultado es correcto.
――――――――――――――――――――――――――――――――――――――――
――――――――――――――――――――――――――――――――――――――――
5_val_1 | Tu resultado es correcto.
――――――――――――――――――――――――――――――――――――――――


## Conclusión

Con estos pasos —registrar las respuestas correctas con `FileAnswer`, exportarlas a un archivo de texto y luego evaluarlas con `Quiz`— es posible automatizar el proceso de calificación en notebooks. La biblioteca de **macti_lib** cuenta con la capacidad de manejar diferentes tipos de preguntas (opción múltiple, expresiones simbólicas, valores numéricos, estructuras de datos y diccionarios), y ofrece retroalimentación directa al estudiante, ahorrando tiempo y asegurando consistencia en la evaluación.
